In [ ]:
import json
import glob
from pathlib import Path
from dotenv import load_dotenv
import unicodedata
import re
import difflib
from openai import OpenAI

video_url_map = {
    "01_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #29 | 과거에 있었던 일 설명할 때  | 기초 영어 회화.json": "https://www.youtube.com/watch?v=R_-pgaQYaYQ",
    "02_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #28 | 마법과 같은 that | 기초 영어 회화.json": "https://www.youtube.com/watch?v=008886a-lQI",
    "03_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 ##27 | 수동태를 that과 연결하기! | 기초 영어 회화.json": "https://www.youtube.com/watch?v=r3qBF9dMz10",
    "04_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #26 | 수동태 핵심 파악! | 기초 영어 회화.json": "https://www.youtube.com/watch?v=bGl_7acUnNk",
    "05_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #25 | have를 깊이, 자연스럽게 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=jOb8mznvX48",
    "06_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #24 | 현재완료 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=RgNbTRRt78Y",
    "07_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #23 | that 뒤에 조동사 쓰기  | 기초 영어 회화.json": "https://www.youtube.com/watch?v=OaTujaboBf8",
    "08_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #22 | that 뒤에 진행형을 가지고 만들어 보자  | 기초 영어 회화.json": "https://www.youtube.com/watch?v=NXbGg9nxpdk",
    "09_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #21 | 또 다른 that | 기초 영어 회화.json": "https://www.youtube.com/watch?v=k3-666q27Ps",
    "10_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #20 | that으로 문장을 길게! (3) | 기초 영어 회화.json": "https://www.youtube.com/watch?v=JMfB_2pfqCA",
    "11_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #19 | that 으로 문장 길게 만들기(2) | 기초 영어 회화.json": "https://www.youtube.com/watch?v=gHrI6qhbziI",
    "12_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#18 | that 으로 문장 길게 만들기(1) | 기초 영어 회화.json": "https://www.youtube.com/watch?v=nHbEN7KEmmE",
    "13_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #17 | can을 be able to로 바꾸기! | 기초 영어 회화.json": "https://www.youtube.com/watch?v=CGN1TdvhkvY",
    "14_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #16 | 동명사로 주어 길게 만들기 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=jzJzdoBdeAc",
    "15_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #15 | 목적을 나타내는 to 동사원형 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=7CvXgPmdD9s",
    "16_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #14 | 미래와 과거에도 진행형을 쓴다! | 기초 영어 회화.json": "https://www.youtube.com/watch?v=7ot7hY8wm4Q",
    "17_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #13 | 현재 진행형의 다양한 쓰임.json": "https://www.youtube.com/watch?v=j87QB9EZZrY",
    "18_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #12 | '~에 있다'를 뜻하는 be 동사 연습 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=h_Yv5bX8p8k",
    "19_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#11 | 다양한 be 동사 형태 연습 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=WS5hLZV7Lb4",
    "20_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#10 | 바쁘다는 busy가 아니다! | 기초 영어 회화.json": "https://www.youtube.com/watch?v=V8e_cwY7VTs",
    "21_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#9 | if로 문장 길게 만들기 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=hh9pAAS-gho",
    "22_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#8 | '~하기를' to 동사원형 연습 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=TK1HL_27g6U",
    "23_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #5  | 의문문 길게 만들기 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=EXKS6rZHbbA",
    "24_NEW 이시원의 기초 영어 회화 강의한 달 만에 영어로 말문 트기 #4 | 과거 시제 마스터 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=G_SNroMhJTQ",
    "25_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#3 | And 로 문장을 길게!  | 기초 영어 회화.json": "https://www.youtube.com/watch?v=VJeidy58uJQ",
    "26_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#1 | 영어는 단어의 연결 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=oLIpoVoDgTo",
    "27_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#2 | 미래를 나타내는 will | 기초 영어 회화.json": "https://www.youtube.com/watch?v=KQbWy6j_TFA",
    "28_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #30 | 많이 쓰는 동사 put / get / take | 기초 영어 회화.json": "https://www.youtube.com/watch?v=1dIALFMvJlA",
    "29_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기#6 | 의문문 질문에 답하기 | 기초 영어 회화.json": "https://www.youtube.com/watch?v=a_eNZ4ZVwxc",
    "30_NEW 이시원 강의 | 한 달 만에 영어로 말문 트기 #7 | 가능과 허락의 can | 기초 영어 회화.json": "https://www.youtube.com/watch?v=kYx8f4U4-jo"
}

def normalize_filename(name):
    # NFC 정규화 (맥/리눅스 파일시스템 호환)
    name = unicodedata.normalize("NFC", name)
    # 공백과 특수문자 통일
    name = name.replace("｜", "|").replace("＃", "#")
    name = re.sub(r"\s+", " ", name.strip())
    return name

import difflib
def get_video_url(video_filename):
    keys = list(video_url_map.keys())
    match = difflib.get_close_matches(video_filename, keys, n=1, cutoff=0.6)
    if match:
        return video_url_map[match[0]]
    return "URL_없음"

In [ ]:
json_files = sorted(glob.glob("./youtube_playlist/*.json"))

all_docs = []

for json_file in json_files:
    video_filename = normalize_filename(os.path.basename(json_file))
    video_url = get_video_url(video_filename)

# OCR 종류
1. 무료+간단 : EsyOCR
2. 무료 + 정확도: PaddleOCR

In [20]:
import cv2
from paddleocr import PaddleOCR

import json
from pathlib import Path

#EasyOCR init (Korean+English)
# PaddleOCR 초기화 (한국어 + 영어)
ocr = PaddleOCR(use_angle_cls=True, lang='korean', use_gpu=False, show_log=False)


def extract_frames(video_path, interval=5):
    """영상에서 N초 간격으로 프레임 추출, 5초먼 될까?"""
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frames = []
    
    frame_interval = int(fps * interval)
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_count % frame_interval == 0:
            timestamp = frame_count / fps
            frames.append({
                "timestamp" : timestamp,
                "frame": frame
            })
            
        frame_count += 1
    cap.release()
    print(f"{len(frames)}개 프레임 추출")
    return frames

a = extract_frames("https://www.youtube.com/watch?v=R_-pgaQYaYQ", interval=5)


RuntimeError: operator torchvision::nms does not exist

In [29]:
import cv2
import base64

# YouTube URL → 직접은 안 되고, 스트림 URL 필요
# 방법 1: yt-dlp로 스트림 URL 얻기

import subprocess
import json

def get_stream_url(youtube_url):
    """YouTube URL에서 스트림 URL 추출"""
    cmd = ['yt-dlp', '-f', 'best', '-g', youtube_url]
    result = subprocess.run(cmd, capture_output=True, text=True)
    return result.stdout.strip()

# 사용
youtube_url = "https://www.youtube.com/watch?v=j87QB9EZZrY"
stream_url = get_stream_url(youtube_url)

# OpenCV로 직접 읽기
cap = cv2.VideoCapture(stream_url)
cap.set(cv2.CAP_PROP_POS_MSEC, 10000)  # 30초
ret, frame = cap.read()
cap.release()


In [30]:
cv2.imwrite('test_frame.jpg', frame)
print("✅ test_frame.jpg 저장됨")

✅ test_frame.jpg 저장됨


In [33]:
import cv2
import numpy as np

def extract_changed_frames(video_path, threshold=30):
    """화면이 바뀔 때만 프레임 추출"""

    stream_url = get_stream_url(video_path)
    
    cap = cv2.VideoCapture(stream_url)
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    frames = []
    prev_frame = None
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # 그레이스케일로 변환
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        if prev_frame is not None:
            # 이전 프레임과 차이 계산
            diff = cv2.absdiff(prev_frame, gray)
            change = np.mean(diff)
            
            # 변화가 크면 저장 (화면 전환)
            if change > threshold:
                timestamp = frame_count / fps
                frames.append({
                    "timestamp": round(timestamp, 2),
                    "frame": frame,
                    "change": round(change, 2)
                })
        
        prev_frame = gray
        frame_count += 1
    
    cap.release()
    print(f"📷 {len(frames)}개 프레임 (화면 변화 감지)")
    return frames
a = extract_changed_frames("https://www.youtube.com/watch?v=j87QB9EZZrY", threshold=30)
a

📷 5개 프레임 (화면 변화 감지)


[{'timestamp': 8.83,
  'frame': array([[[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         ...,
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         [[255, 255, 255],
          [255, 255, 255],
          [255, 255, 255],
          ...,
          [255, 255, 255],
          [255, 255, 255],
          [255, 255, 255]],
  
         [[255, 255, 255],
          [255, 255, 

In [36]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI()

def test_vision(frame):
    _, buffer = cv2.imencode('.jpg', frame)
    base64_image = base64.b64encode(buffer).decode('utf-8')
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{
            "role": "user",
            "content": [
                {"type": "text", "text": "이 영어 강의 화면의 텍스트를 모두 추출하세요. JSON으로 출력."},
                {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
            ]
        }],
        response_format={"type": "json_object"},
        max_tokens=500
    )
    return json.loads(response.choices[0].message.content)
# a는 리스트, a[0]은 첫 번째 딕셔너리, a[0]["frame"]이 이미지
print(f"총 {len(a)}개 프레임")

# 첫 번째 프레임으로 테스트
frame = a[0]["frame"]
print("frame shape:", frame.shape)

# Vision 테스트
result = test_vision(frame)
print(result)



총 5개 프레임
frame shape: (1080, 1920, 3)


TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [ ]:
import base64
import cv2
import openai
from dotenv import load_dotenv

load_dotenv()
client = openai()

def extract_text_with_vision(frame):
    """GPT-4o Vision으로 화면 분석"""
    
    # 프레임을 base64로 변환
    _, buffer = cv2.imencode('.jpg', frame)
    base64_image = base64.b64encode(buffer).decode('utf-8')
    
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[{
            "role": "user",
            "content": [
                {
                    "type": "text", 
                    "text": """이 영어 강의 화면을 분석하세요.

                        JSON으로 출력:
                        {
                        "title": "화면 제목/주제",
                        "main_text": ["핵심 텍스트들"],
                        "examples": ["예문들"],
                        "notes": "기타 메모"
                        }"""
                },
                {
                    "type": "image_url", 
                    "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
                }
            ]
        }],
        response_format={"type": "json_object"},
        max_tokens=500
    )
    
    return json.loads(response.choices[0].message.content)
